<a href="https://colab.research.google.com/github/carlosvalenciano/Project4/blob/main/Stock_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Requirements
!pip install yahoo_fin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=4cb4a2d515df9e8056af2a317ed3f9221a124ee2bbe88a78de0fbeaffaa1730e
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=44bb39736a5008723695ec72ad54a7c073cf9a06251aa9d22fb2539a4c8c6b99
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built bs4 sgmllib3k
  Attempting uninstall: urllib3
    Found existi

In [2]:
import numpy as np
import time as tm
import datetime as dt
import tensorflow as tf
import pandas as pd

# Data preparation
from yahoo_fin import stock_info as yf

In [3]:
# SETTINGS

# Stock ticker, GOOGL
STOCK = input('Enter a ticker. ')
print(f"You entered {STOCK}.")


# Current date
date_now = tm.strftime('%Y-%m-%d')
date_4_years_back = (dt.date.today() - dt.timedelta(days=1460)).strftime('%Y-%m-%d')

Enter a ticker. META
You entered META.


In [4]:
# LOAD DATA
# from yahoo_fin
# for 1104 bars with interval = 1d (one day)
df = yf.get_data(
    STOCK,
    start_date=date_4_years_back,
    end_date=date_now,
    interval='1d')

In [5]:
df

,open,high,low,close,adjclose,volume,ticker
2019-12-16,195.270004,199.119995,194.809998,197.919998,197.919998,14911500,META
2019-12-17,198.839996,199.000000,196.130005,198.389999,198.389999,10175700,META
2019-12-18,200.089996,204.300003,200.089996,202.500000,202.500000,23274400,META
2019-12-19,202.779999,206.300003,202.509995,206.059998,206.059998,16514600,META
2019-12-20,207.479996,207.710007,203.429993,206.300003,206.300003,27306900,META
...,...,...,...,...,...,...,...
2023-12-08,323.089996,333.170013,323.000000,332.750000,332.750000,14077500,META
2023-12-11,329.399994,329.890015,320.000000,325.279999,325.279999,25802500,META
2023-12-12,324.600006,334.470001,324.559998,334.220001,334.220001,18485500,META
2023-12-13,333.929993,338.369995,332.640015,334.739990,334.739990,16353300,META


In [6]:
# Find the relative daily price and the volume for the prior day
df_filter = df["adjclose"].pct_change() * 100
df_filter = df_filter.rename("Today")
df_filter = df_filter.reset_index()
df_filter["Volume"] = df["volume"].shift(1).values / 1000_000_000

In [7]:
# View dataframe
df_filter

,index,Today,Volume
0,2019-12-16,NaN,NaN
1,2019-12-17,0.237470,0.014911
2,2019-12-18,2.071677,0.010176
3,2019-12-19,1.758023,0.023274
4,2019-12-20,0.116474,0.016515
...,...,...,...
1002,2023-12-08,1.886158,0.015905
1003,2023-12-11,-2.244929,0.014077
1004,2023-12-12,2.748402,0.025802
1005,2023-12-13,0.155583,0.018485


In [8]:
# Find the stock prices for the last five days
for i in range(1,6):
    df_filter["Lag " + str(i)] = df_filter["Today"].shift(i)

df_filter

,index,Today,Volume,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
0,2019-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-17,0.237470,0.014911,NaN,NaN,NaN,NaN,NaN
2,2019-12-18,2.071677,0.010176,0.237470,NaN,NaN,NaN,NaN
3,2019-12-19,1.758023,0.023274,2.071677,0.237470,NaN,NaN,NaN
4,2019-12-20,0.116474,0.016515,1.758023,2.071677,0.237470,NaN,NaN
...,...,...,...,...,...,...,...,...
1002,2023-12-08,1.886158,0.015905,2.879188,-0.263909,-0.540585,-1.477747,-0.712207
1003,2023-12-11,-2.244929,0.014077,1.886158,2.879188,-0.263909,-0.540585,-1.477747
1004,2023-12-12,2.748402,0.025802,-2.244929,1.886158,2.879188,-0.263909,-0.540585
1005,2023-12-13,0.155583,0.018485,2.748402,-2.244929,1.886158,2.879188,-0.263909


In [9]:
# Drop the na
df_filter = df_filter.dropna()
df_filter

,index,Today,Volume,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
6,2019-12-24,-0.514113,0.013349,-0.058173,0.116474,1.758023,2.071677,0.237470
7,2019-12-26,1.301676,0.006046,-0.514113,-0.058173,0.116474,1.758023,2.071677
8,2019-12-27,0.149195,0.009351,1.301676,-0.514113,-0.058173,0.116474,1.758023
9,2019-12-30,-1.773187,0.010284,0.149195,1.301676,-0.514113,-0.058173,0.116474
10,2019-12-31,0.410937,0.010524,-1.773187,0.149195,1.301676,-0.514113,-0.058173
...,...,...,...,...,...,...,...,...
1002,2023-12-08,1.886158,0.015905,2.879188,-0.263909,-0.540585,-1.477747,-0.712207
1003,2023-12-11,-2.244929,0.014077,1.886158,2.879188,-0.263909,-0.540585,-1.477747
1004,2023-12-12,2.748402,0.025802,-2.244929,1.886158,2.879188,-0.263909,-0.540585
1005,2023-12-13,0.155583,0.018485,2.748402,-2.244929,1.886158,2.879188,-0.263909


In [10]:
# Find the price change direction
df_filter["Direction"] = [1 if i > 0 else 0 for i in df_filter["Today"]]
df_filter

<ipython-input-10-1bb73288e3f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter["Direction"] = [1 if i > 0 else 0 for i in df_filter["Today"]]


,index,Today,Volume,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Direction
6,2019-12-24,-0.514113,0.013349,-0.058173,0.116474,1.758023,2.071677,0.237470,0
7,2019-12-26,1.301676,0.006046,-0.514113,-0.058173,0.116474,1.758023,2.071677,1
8,2019-12-27,0.149195,0.009351,1.301676,-0.514113,-0.058173,0.116474,1.758023,1
9,2019-12-30,-1.773187,0.010284,0.149195,1.301676,-0.514113,-0.058173,0.116474,0
10,2019-12-31,0.410937,0.010524,-1.773187,0.149195,1.301676,-0.514113,-0.058173,1
...,...,...,...,...,...,...,...,...,...
1002,2023-12-08,1.886158,0.015905,2.879188,-0.263909,-0.540585,-1.477747,-0.712207,1
1003,2023-12-11,-2.244929,0.014077,1.886158,2.879188,-0.263909,-0.540585,-1.477747,0
1004,2023-12-12,2.748402,0.025802,-2.244929,1.886158,2.879188,-0.263909,-0.540585,1
1005,2023-12-13,0.155583,0.018485,2.748402,-2.244929,1.886158,2.879188,-0.263909,1


In [11]:
# Assign the variables: X to Lag 1, Lag 2, Lag 3, Lag 4, Lag 5, Volume and y to Direction
# Set the first 550 for training and last 550 for testing
from sklearn.linear_model import LogisticRegression
X = df_filter[["Lag 1", "Lag 2", "Lag 3", "Lag 4", "Lag 5", "Volume"]]
y = df_filter[["Direction"]]

X_train = X[0:550]
X_test = X[550:]

y_train = y[0:550]
y_test = y[550:]

model = LogisticRegression().fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
# Show the predicted accuracy
model.score(X_test, y_test)

0.5099778270509978

In [13]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_filter['Direction']

# Separate the X variable, the features
X = df_filter.drop(columns=['Direction', 'index', 'Today'])

In [14]:
# Review the y variable Series
y.head()

6     0
7     1
8     1
9     0
10    1
Name: Direction, dtype: int64

In [15]:
# Review the X variable DataFrame
X.head()

,Volume,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
6,0.013349,-0.058173,0.116474,1.758023,2.071677,0.237470
7,0.006046,-0.514113,-0.058173,0.116474,1.758023,2.071677
8,0.009351,1.301676,-0.514113,-0.058173,0.116474,1.758023
9,0.010284,0.149195,1.301676,-0.514113,-0.058173,0.116474
10,0.010524,-1.773187,0.149195,1.301676,-0.514113,-0.058173


In [16]:
# Check the balance of our target values
y.value_counts()

1    517
0    484
Name: Direction, dtype: int64

In [17]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(random_state=42)


# Fit the model using training data
lr_model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [19]:
# Make a prediction using the testing data
testing_predictions = lr_model.predict(X_test)

In [20]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, testing_predictions)

0.4921775223499361

In [21]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, testing_predictions)

array([[ 20, 115],
       [ 19,  97]])

In [22]:
# Print the classification report for the model
target_names = ["Down", "Up"]
testing_report = classification_report(y_test, testing_predictions, target_names=target_names)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

        Down       0.51      0.15      0.23       135
          Up       0.46      0.84      0.59       116

    accuracy                           0.47       251
   macro avg       0.49      0.49      0.41       251
weighted avg       0.49      0.47      0.40       251

